# Regression Week 3: Assessing Fit (polynomial regression)

In this notebook you will compare different regression models in order to assess which model fits best. We will be using polynomial regression as a means to examine this topic. In particular you will:
* Write a function to take an SArray and a degree and return an SFrame where each column is the SArray to a polynomial value up to the total degree e.g. degree = 3 then column 1 is the SArray column 2 is the SArray squared and column 3 is the SArray cubed
* Use matplotlib to visualize polynomial regressions
* Use matplotlib to visualize the same polynomial degree on different subsets of the data
* Use a validation set to select a polynomial degree
* Assess the final fit using test data

We will continue to use the House data from previous notebooks.

# Fire up graphlab create

In [1]:
import graphlab

Next we're going to write a polynomial function that takes an SArray and a maximal degree and returns an SFrame with columns containing the SArray to all the powers up to the maximal degree.

The easiest way to apply a power to an SArray is to use the .apply() and lambda x: functions. 
For example to take the example array and compute the third power we can do as follows: (note running this cell the first time may take longer than expected since it loads graphlab)

In [2]:
tmp = graphlab.SArray([1., 2., 3.])
tmp_cubed = tmp.apply(lambda x: x**3)
print tmp
print tmp_cubed

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1517352211.log


This non-commercial license of GraphLab Create for academic use is assigned to baiganovm@gmail.com and will expire on November 13, 2018.


Less than 8 successfully started. Using only 4 workers.

All operations will proceed as normal, but lambda operations will not be able to use all available cores.

(The location of the log file is printed at the start of the GraphLab server).

[1.0, 2.0, 3.0]
[1.0, 8.0, 27.0]


We can create an empty SFrame using graphlab.SFrame() and then add any columns to it with ex_sframe['column_name'] = value. For example we create an empty SFrame and make the column 'power_1' to be the first power of tmp (i.e. tmp itself).

In [3]:
ex_sframe = graphlab.SFrame()
ex_sframe['power_1'] = tmp
print ex_sframe

+---------+
| power_1 |
+---------+
|   1.0   |
|   2.0   |
|   3.0   |
+---------+
[3 rows x 1 columns]



# Polynomial_sframe function

Using the hints above complete the following function to create an SFrame consisting of the powers of an SArray up to a specific degree:

In [4]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature ** power
    return poly_sframe

To test your function consider the smaller tmp variable and what you would expect the outcome of the following call:

In [5]:
print polynomial_sframe(tmp, 3)

+---------+---------+---------+
| power_1 | power_2 | power_3 |
+---------+---------+---------+
|   1.0   |   1.0   |   1.0   |
|   2.0   |   4.0   |   8.0   |
|   3.0   |   9.0   |   27.0  |
+---------+---------+---------+
[3 rows x 3 columns]



# Visualizing polynomial regression

Let's use matplotlib to visualize what a polynomial regression looks like on some real data.

In [6]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [7]:
sales = sales.sort(['sqft_living', 'price'])

Let's start with a degree 1 polynomial using 'sqft_living' (i.e. a line) to predict 'price' and plot what it looks like.

In [8]:
poly1_data = polynomial_sframe(sales['sqft_living'], 1)
poly1_data['price'] = sales['price'] # add price to the data since it's the target

NOTE: for all the models in this notebook use validation_set = None to ensure that all results are consistent across users.

In [9]:
model1 = graphlab.linear_regression.create(poly1_data, target = 'price', features = ['power_1'], validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 1

Number of unpacked features : 1

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.032233     | 4362074.696077     | 261440.790724 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [10]:
#let's take a look at the weights before we plot
model1.get("coefficients")

name,index,value,stderr
(intercept),None,-43579.0852515,4402.68969743
power_1,None,280.622770886,1.93639855513


In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

AttributeError: 'module' object has no attribute 'cbook'

In [12]:
plt.plot(poly1_data['power_1'],poly1_data['price'],'.',
        poly1_data['power_1'], model1.predict(poly1_data),'-')

NameError: name 'plt' is not defined

Let's unpack that plt.plot() command. The first pair of SArrays we passed are the 1st power of sqft and the actual price we then ask it to print these as dots '.'. The next pair we pass is the 1st power of sqft and the predicted values from the linear model. We ask these to be plotted as a line '-'. 

We can see, not surprisingly, that the predicted values all fall on a line, specifically the one with slope 280 and intercept -43579. What if we wanted to plot a second degree polynomial?

In [13]:
poly2_data = polynomial_sframe(sales['sqft_living'], 2)
my_features = poly2_data.column_names() # get the name of the features
poly2_data['price'] = sales['price'] # add price to the data since it's the target
model2 = graphlab.linear_regression.create(poly2_data, target = 'price', features = my_features, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 2

Number of unpacked features : 2

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.031002     | 5913020.984255     | 250948.368758 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [14]:
model2.get("coefficients")

name,index,value,stderr
(intercept),None,199222.496445,7058.00483552
power_1,None,67.9940640677,5.28787201316
power_2,None,0.0385812312789,0.000898246547032


In [15]:
plt.plot(poly2_data['power_1'],poly2_data['price'],'.',
        poly2_data['power_1'], model2.predict(poly2_data),'-')

NameError: name 'plt' is not defined

The resulting model looks like half a parabola. Try on your own to see what the cubic looks like:

In [16]:
poly3_data = polynomial_sframe(sales['sqft_living'], 3)
my_features = poly3_data.column_names() # get the name of the features
poly3_data['price'] = sales['price'] # add price to the data since it's the target
model3 = graphlab.linear_regression.create(poly3_data, target = 'price', features = my_features, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.051052     | 3261066.736007     | 249261.286346 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [17]:
model3.get("coefficients")

name,index,value,stderr
(intercept),None,336788.117952,10661.0153713
power_1,None,-90.1476236119,10.6222891844
power_2,None,0.087036715081,0.00296630623148
power_3,None,-3.8398521196e-06,2.2417490959e-07


In [18]:
plt.plot(poly3_data['power_1'],poly3_data['price'],'.',
        poly3_data['power_1'], model3.predict(poly3_data),'-')

NameError: name 'plt' is not defined

Now try a 15th degree polynomial:

In [19]:
poly15_data = polynomial_sframe(sales['sqft_living'], 15)
my_features = poly15_data.column_names() # get the name of the features
poly15_data['price'] = sales['price'] # add price to the data since it's the target
model15 = graphlab.linear_regression.create(poly15_data, target = 'price', features = my_features, validation_set = None)
model15.get("coefficients").print_rows(num_rows=16)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.022452     | 2662308.584344     | 245690.511190 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   73619.7521179    |        nan        |
|   power_1   |  None |   410.287462517    |        nan        |
|   power_2   |  None |  -0.230450714409   |        nan        |
|   power_3   |  None | 7.58840542349e-05  |        nan        |
|   power_4   |  None | -5.65701802386e-09 |        nan        |
|   power_5   |  None | -4.57028130967e-13 |        nan        |
|   power_6   |  None | 2.66360206727e-17  |        nan        |
|   power_7   |  None | 3.38584769163e-21  |        nan        |
|   power_8   |  None |  1.147231041e-25   |        nan        |
|   power_9   |  None | -4.65293586061e-30 |        nan        |
|   power_10  |  None | -8.6879620195e-34  | 1.98250666775e-29 |
|   power_11  |  None | -6.30994295678e-38 |        nan        |
|   power_12  |  None | -

In [20]:
plt.plot(poly15_data['power_1'],poly15_data['price'],'.',
        poly15_data['power_1'], model15.predict(poly3_data),'-')

NameError: name 'plt' is not defined

What do you think of the 15th degree polynomial? Do you think this is appropriate? If we were to change the data do you think you'd get pretty much the same curve? Let's take a look.

# Changing the data and re-learning

We're going to split the sales data into four subsets of roughly equal size. Then you will estimate a 15th degree polynomial model on all four subsets of the data. Print the coefficients (you should use .print_rows(num_rows = 16) to view all of them) and plot the resulting fit (as we did above). The quiz will ask you some questions about these results.

To split the sales data into four subsets, we perform the following steps:
* First split sales into 2 subsets with `.random_split(0.5, seed=0)`. 
* Next split the resulting subsets into 2 more subsets each. Use `.random_split(0.5, seed=0)`.

We set `seed=0` in these steps so that different users get consistent results.
You should end up with 4 subsets (`set_1`, `set_2`, `set_3`, `set_4`) of approximately equal size. 

In [21]:
(test_data, train_data) = sales.random_split(0.5, seed=0)
(set_1, set_2) = test_data.random_split(0.5, seed=0)
(set_3, set_4) = train_data.random_split(0.5, seed=0)

Fit a 15th degree polynomial on set_1, set_2, set_3, and set_4 using sqft_living to predict prices. Print the coefficients and make a plot of the resulting model.

In [22]:
poly_data_set1 = polynomial_sframe(set_1['sqft_living'], 15)
set_1_features = poly_data_set1.column_names() # get the name of the features
poly_data_set1['price'] = set_1['price'] # add price to the data since it's the target
set1_model = graphlab.linear_regression.create(poly_data_set1, target = 'price', features = set_1_features, validation_set = None)
set1_model.get("coefficients").print_rows(num_rows=16)

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.025206     | 2195218.932304     | 248858.822200 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |    223312.75025    |   720102.105813   |
|   power_1   |  None |   118.086127586    |   2991.25004237   |
|   power_2   |  None |  -0.0473482011336  |   5.14643044195   |
|   power_3   |  None | 3.25310342468e-05  |  0.00486477316272 |
|   power_4   |  None | -3.3237215256e-09  | 2.83839733901e-06 |
|   power_5   |  None | -9.75830457822e-14 | 1.09654745207e-09 |
|   power_6   |  None | 1.15440303425e-17  | 2.97349100848e-13 |
|   power_7   |  None | 1.05145869431e-21  | 5.95020515282e-17 |
|   power_8   |  None | 3.46049616301e-26  | 8.94450727081e-21 |
|   power_9   |  None | -1.09654454076e-30 |  1.0057647634e-24 |
|   power_10  |  None | -2.42031812181e-34 | 8.73926154652e-29 |
|   power_11  |  None | -1.99601206791e-38 | 4.36576035029e-33 |
|   power_12  |  None | -

In [23]:
poly_data_set2 = polynomial_sframe(set_2['sqft_living'], 15)
set_2_features = poly_data_set2.column_names() # get the name of the features
poly_data_set2['price'] = set_2['price'] # add price to the data since it's the target
set2_model = graphlab.linear_regression.create(poly_data_set2, target = 'price', features = set_2_features, validation_set = None)
set2_model.get("coefficients").print_rows(num_rows=16)

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.024424     | 2069212.978547     | 234840.067186 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   89836.5077348    |   1425584.05986   |
|   power_1   |  None |    319.80694676    |   7890.23598628   |
|   power_2   |  None |  -0.103315397038   |   18.4602850363   |
|   power_3   |  None | 1.06682476058e-05  |  0.0241806306245  |
|   power_4   |  None | 5.75577097729e-09  | 1.98048020398e-05 |
|   power_5   |  None | -2.5466346474e-13  | 1.06980818362e-08 |
|   power_6   |  None | -1.09641345066e-16 | 3.89653322973e-12 |
|   power_7   |  None | -6.36458441707e-21 | 9.52793175493e-16 |
|   power_8   |  None | 5.52560416968e-25  | 1.49691897587e-19 |
|   power_9   |  None |  1.3508203898e-28  | 1.24414181467e-23 |
|   power_10  |  None | 1.18408188241e-32  |        nan        |
|   power_11  |  None | 1.98348000462e-37  | 1.49946194435e-31 |
|   power_12  |  None | -

In [24]:
poly_data_set3 = polynomial_sframe(set_3['sqft_living'], 15)
set_3_features = poly_data_set3.column_names() # get the name of the features
poly_data_set3['price'] = set_3['price'] # add price to the data since it's the target
set1_mode3 = graphlab.linear_regression.create(poly_data_set3, target = 'price', features = set_3_features, validation_set = None)
set1_mode3.get("coefficients").print_rows(num_rows=16)

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.025624     | 2269769.506523     | 251460.072754 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |    87317.97956     |   1265937.15642   |
|   power_1   |  None |   356.304911031    |   6169.20706961   |
|   power_2   |  None |  -0.164817442795   |   12.6758534148   |
|   power_3   |  None | 4.40424992635e-05  |   0.014531934999  |
|   power_4   |  None | 6.48234877396e-10  | 1.03509477362e-05 |
|   power_5   |  None | -6.75253226641e-13 | 4.80452542019e-09 |
|   power_6   |  None | -3.36842592784e-17 |  1.4688465776e-12 |
|   power_7   |  None | 3.60999704377e-21  | 2.86648472244e-16 |
|   power_8   |  None | 6.46999725636e-25  | 3.17165326664e-20 |
|   power_9   |  None | 4.23639388651e-29  |  2.0816121888e-24 |
|   power_10  |  None | -3.62149423631e-34 | 4.62448511879e-28 |
|   power_11  |  None | -4.27119527371e-37 | 5.18893736143e-32 |
|   power_12  |  None | -

In [25]:
poly_data_set4 = polynomial_sframe(set_4['sqft_living'], 15)
set_4_features = poly_data_set4.column_names() # get the name of the features
poly_data_set4['price'] = set_4['price'] # add price to the data since it's the target
set4_model = graphlab.linear_regression.create(poly_data_set4, target = 'price', features = set_4_features, validation_set = None)
set4_model.get("coefficients").print_rows(num_rows=16)

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.022199     | 2314893.173824     | 244563.136754 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   259020.879447    |   5299947.80822   |
|   power_1   |  None |   -31.7277161932   |   37868.2949634   |
|   power_2   |  None |   0.109702769609   |   109.794778973   |
|   power_3   |  None | -1.58383847314e-05 |   0.173877488964  |
|   power_4   |  None | -4.47660623787e-09 |  0.00016956349828 |
|   power_5   |  None | 1.13976573478e-12  | 1.08145862046e-07 |
|   power_6   |  None | 1.97669120543e-16  | 4.64434081715e-11 |
|   power_7   |  None | -6.15783678607e-21 |  1.350569008e-14  |
|   power_8   |  None | -4.88012304096e-24 | 2.60220331641e-18 |
|   power_9   |  None | -6.6218678116e-28  | 3.08677179769e-22 |
|   power_10  |  None | -2.70631583575e-32 | 1.78880431884e-26 |
|   power_11  |  None | 6.72370411717e-36  | 1.03141539371e-30 |
|   power_12  |  None | 1

Some questions you will be asked on your quiz:

**Quiz Question: Is the sign (positive or negative) for power_15 the same in all four models?**

**Quiz Question: (True/False) the plotted fitted lines look the same in all four plots**

# Selecting a Polynomial Degree

Whenever we have a "magic" parameter like the degree of the polynomial there is one well-known way to select these parameters: validation set. (We will explore another approach in week 4).

We split the sales dataset 3-way into training set, test set, and validation set as follows:

* Split our sales data into 2 sets: `training_and_validation` and `testing`. Use `random_split(0.9, seed=1)`.
* Further split our training data into two sets: `training` and `validation`. Use `random_split(0.5, seed=1)`.

Again, we set `seed=1` to obtain consistent results for different users.

In [26]:
(training_and_validation, testing) = sales.random_split(0.9, seed=1)
(training, validation) = training_and_validation.random_split(0.5, seed=1)

Next you should write a loop that does the following:
* For degree in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] (to get this in python type range(1, 15+1))
    * Build an SFrame of polynomial data of train_data['sqft_living'] at the current degree
    * hint: my_features = poly_data.column_names() gives you a list e.g. ['power_1', 'power_2', 'power_3'] which you might find useful for graphlab.linear_regression.create( features = my_features)
    * Add train_data['price'] to the polynomial SFrame
    * Learn a polynomial regression model to sqft vs price with that degree on TRAIN data
    * Compute the RSS on VALIDATION data (here you will want to use .predict()) for that degree and you will need to make a polynmial SFrame using validation data.
* Report which degree had the lowest RSS on validation data (remember python indexes from 0)

(Note you can turn off the print out of linear_regression.create() with verbose = False)

In [28]:
rss_list = []
for i in range(1,15+1):
    train_data = polynomial_sframe(training['sqft_living'], i)
    features = train_data.column_names()
    train_data['price'] = training['price']
    model = graphlab.linear_regression.create(train_data, target = 'price', features = features, validation_set = None, verbose = False)

    val_data = polynomial_sframe(validation['sqft_living'], i)
    predictions = model.predict(val_data)
    errors = validation['price'] - predictions
    rss = (errors**2).sum()
    rss_list.append(rss)
    print('Mdl pwr: ' + str(i) + ' RSS is ' + str(rss))
    model.get("coefficients").print_rows(num_rows=16)



Mdl pwr: 1 RSS is 6.76709775198e+14
+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -45052.3457811 | 6527.16126153 |
|   power_1   |  None | 282.175955956  |  2.8806101547 |
+-------------+-------+----------------+---------------+
[2 rows x 4 columns]

Mdl pwr: 2 RSS is 6.07090530698e+14
+-------------+-------+-----------------+------------------+
|     name    | index |      value      |      stderr      |
+-------------+-------+-----------------+------------------+
| (intercept) |  None |  147822.568657  |  10320.6419393   |
|   power_1   |  None |  115.415214461  |  7.57323236995   |
|   power_2   |  None | 0.0296646114018 | 0.00125162597797 |
+-------------+-------+-----------------+------------------+
[3 rows x 4 columns]

Mdl pwr: 3 RSS is 6.16714574533e+14
+-------------+-------+--------------------+-------------------+
|     name    

**Quiz Question: Which degree (1, 2, …, 15) had the lowest RSS on Validation data?**

Now that you have chosen the degree of your polynomial using validation data, compute the RSS of this model on TEST data. Report the RSS on your quiz.

RSS is 5.8918247781e+14


**Quiz Question: what is the RSS on TEST data for the model with the degree selected from Validation data?**

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   206520.746109    |    62365.43629    |
|   power_1   |  None |   191.386412848    |   124.992902904   |
|   power_2   |  None |  -0.141145435547   |  0.0912790132821  |
|   power_3   |  None | 8.58496330317e-05  | 3.10860683014e-05 |
|   power_4   |  None | -1.77908642189e-08 |  5.2331831722e-09 |
|   power_5   |  None | 1.65930833699e-12  | 4.16262614494e-13 |
|   power_6   |  None | -5.59634436006e-17 | 1.22955192237e-17 |
+-------------+-------+--------------------+-------------------+